In [172]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np


In [173]:
# 将数据集使用滑动窗口方法制作成监督学习
def series_to_supervised(df,n_in=96, n_out=96):   #传入的data应当是一个frame
    n_vars = df.shape[1]         #获取变量数
    df=DataFrame(df)
    cols, names = [], []
    #i: n_in, n_in-1, ..., 1，为滞后期数
    #分别代表t-n_in, ... ,t-1期
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    #i: 0, 1, ..., n_out-1，为超前预测的期数
    #分别代表t，t+1， ... ,t+n_out-1期
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names

    return agg

In [174]:
#读取数据集
dataset_train=pd.read_csv("train_set.csv",infer_datetime_format=True,index_col='date')    #设置时间戳索引
dataset_test=pd.read_csv("test_set.csv",infer_datetime_format=True,index_col='date')    #设置时间戳索引
dataset_val=pd.read_csv("validation_set.csv",infer_datetime_format=True,index_col='date')    #设置时间戳索引


values_train = dataset_train.values
values_test = dataset_test.values
values_val = dataset_val.values



C:\Users\12204\AppData\Local\Temp\ipykernel_22020\1782723478.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset_train=pd.read_csv("train_set.csv",infer_datetime_format=True,index_col='date')    #设置时间戳索引
C:\Users\12204\AppData\Local\Temp\ipykernel_22020\1782723478.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset_test=pd.read_csv("test_set.csv",infer_datetime_format=True,index_col='date')    #设置时间戳索引
C:\Users\12204\AppData\Local\Temp\ipykernel_22020\1782723478.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and

In [175]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(values_train)
scaled_test = scaler.fit_transform(values_test)
scaled_val = scaler.fit_transform(values_val)

# scale         #归一化

In [176]:
reframed_train = series_to_supervised(scaled_train, 96, 96)
reframed_test = series_to_supervised(scaled_test, 96, 96)
reframed_val = series_to_supervised(scaled_val, 96, 96)


reframed_train.shape

(8640, 1344)

In [177]:
# reframed = reframed[:]
# reframed.shape

In [178]:
n_in = 96
n_out=96
n_vars=7
col_names = ['HUFL','HULL','MUFL','MULL','LUFL','LULL','OT']
contain_vars = []
for i in range(n_in,0,-1):
    contain_vars += [('%s(t-%d)' % (col_names[j], i)) for j in range(0,n_vars)]  
for i in range(0, n_out):
    contain_vars += [('%s(t+%d)' % (col_names[j], i)) for j in range(0,n_vars)]  

len(contain_vars)

reframed_train.columns=contain_vars
reframed_test.columns=contain_vars
reframed_val.columns=contain_vars

import copy
data_train = copy.deepcopy(reframed_train)
data_test = copy.deepcopy(reframed_test)
data_val = copy.deepcopy(reframed_val)


    #修改列名
    # contain_vars = []
    # for i in range(1,n_in+1):
    #     contain_vars += [('%s(t-%d)' % (col_names[i], j)) for j in range(n_vars)]
    # for i in range(n_vars):
    #     contain_vars += [('%s(t+%d)' % (col_names[i], j)) for j in range(0,n_out-1)]   
    # data.columns = contain_vars +  ['Y(t)'] + [('Y(t+%d)' % (j)) for j in range(1,n_out)]
    # #分隔数据集，分为训练集和测试集
    # values = data.values

In [179]:
data_train.shape

(8640, 1344)

In [180]:
pd.set_option('display.max_columns', None)
trainn = data_train.values
vall = data_val.values
testt = data_test.values

# data.shape
# data.values
# values=data.values
# data
# n_train = round(data.shape[0]*0.6)
# n_val = round(data.shape[0]*0.2)
# trainn = values[:n_train, :]
# vall = values[n_train:n_train+n_val, :]
# testt = values[n_train+n_val:, :]





train_X, train_y = trainn[:, :n_in*n_vars], trainn[:, n_in*n_vars:]
val_X, val_y = vall[:, :n_in*n_vars], vall[:, n_in*n_vars:]
test_X, test_y = testt[:, :n_in*n_vars], testt[:, n_in*n_vars:]

train_X = train_X.reshape((train_X.shape[0], n_in, n_vars))
# train_y = train_y.reshape((train_y.shape[0], n_in, n_vars))

val_X = val_X.reshape((val_X.shape[0], n_in, n_vars))
# val_y= val_y.reshape((val_y.shape[0], n_in, n_vars))

test_X =test_X.reshape((test_X.shape[0], n_in, n_vars))
# test_y =test_y.reshape((test_X.shape[0], n_in, n_vars))


#将输入X改造为LSTM的输入格式，即[samples,timesteps,features]
# train_X = train_X.reshape((train_X.shape[0], n_in, n_vars))
# test_X = test_X.reshape((test_X.shape[0], n_in, n_vars))
# return scaler, data, train_X, train_y, test_X, test_y, dataset


In [181]:

train_X.shape
trainn.shape
train_X[-1]

array([[0.66981935, 0.42073918, 0.7378191 , 0.61890871, 0.40266608,
        0.13108446, 0.44101265],
       [0.70300487, 0.47540182, 0.77183763, 0.61281744, 0.40266608,
        0.15870501, 0.4395951 ],
       [0.68246144, 0.46993557, 0.75345622, 0.61281744, 0.38250522,
        0.15168668, 0.43398486],
       [0.6461154 , 0.41527292, 0.73411693, 0.58845229, 0.3724799 ,
        0.15168668, 0.43398486],
       [0.69984435, 0.48086808, 0.7709315 , 0.65245367, 0.36245456,
        0.15168668, 0.41853174],
       [0.69194303, 0.46446929, 0.76541708, 0.63417983, 0.37920017,
        0.15168668, 0.43116978],
       [0.75038918, 0.46993557, 0.79759747, 0.63109129, 0.49663985,
        0.21394613, 0.44382773],
       [0.71090619, 0.45353676, 0.75529437, 0.54881606, 0.46975873,
        0.21394613, 0.44662289],
       [0.66191804, 0.44260424, 0.70651893, 0.58227523, 0.4933348 ,
        0.2621689 , 0.46067841],
       [0.41388745, 0.57926085, 0.43698545, 0.70118394, 0.48661453,
        0.27597918, 0.4

In [182]:
def fit_lstm(train_Xx,train_yy,val_Xx,val_yy, n_neurons=50, n_batch=72, n_epoch=10, loss='mae', optimizer='adam', repeats=1):
    train_X = train_Xx
    train_y = train_yy
    val_X = val_Xx
    val_y = val_yy

    model_list = []
    for i in range(repeats):
        #设计神经网络
        model = Sequential()
        model.add(LSTM(n_neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(train_y.shape[1]))
        model.compile(loss=loss, optimizer=optimizer)
        #拟合神经网络
        history = model.fit(train_X, train_y, epochs=n_epoch, batch_size=n_batch, validation_data=(val_X, val_y), verbose=0, shuffle=False)
        #画出学习过程
        p1 = pyplot.plot(history.history['loss'], color='blue', label='train')
        p2 = pyplot.plot(history.history['val_loss'], color='yellow',label='val')
        #保存model
        model_list.append(model)
    pyplot.legend(["train","test"])
    pyplot.show()
    return model_list

In [183]:
def lstm_predict(model, scaler,test_X,test_y):
    # 做出预测
    yhat = model.predict(test_X)
    #将测试集上的预测值还原为原来的数据维度
    scale_new = MinMaxScaler()
    scale_new.min_, scale_new.scale_ = scaler.min_[0], scaler.scale_[0]
    inv_yhat = scale_new.inverse_transform(yhat)
    #将测试集上的实际值还原为原来的数据维度
    inv_y = scale_new.inverse_transform(test_y)
    return inv_yhat, inv_y

In [184]:
def evaluate_forecasts(test, forecasts, n_out):
    rmse_dic = {}
    for i in range(n_out):
        actual = [float(row[i]) for row in test]
        predicted = [float(forecast[i]) for forecast in forecasts]
        rmse = sqrt(mean_squared_error(actual, predicted))
        rmse_dic['t+' + str(i+1) + ' RMSE'] = rmse
    return rmse_dic

In [185]:
#以原始数据为背景画出预测数据
def plot_forecasts(series, forecasts):
    #用蓝色画出原始数据集
    pyplot.plot(series.values)
    n_seq = len(forecasts[0])
    #用红色画出预测值
    for i in range(1,len(forecasts)+1):
        xaxis = [x for x in range(i, i+n_seq+1)]
        yaxis = [float(series.iloc[i-1,0])] + list(forecasts[i-1])
        pyplot.plot(xaxis, yaxis, color='red')
    #展示图像
    pyplot.show()

In [186]:
train_X.shape

(8640, 96, 7)

In [187]:
train_y=train_y.reshape(train_y.shape[0],96,7)
val_y=val_y.reshape(val_y.shape[0],96,7)
train_y.shape
                  
train_X = train_X[96:]
train_y = train_y[:-96]
val_X=val_X[96:]
val_y = val_y[:-96]
# train_y[-96:]



In [188]:
train_X.shape
train_y[-1]
# train_y.shape
# test_X.shape
val_y

array([[[0.72584233, 0.48197713, 0.7871379 , ..., 0.21111838,
         0.14053173, 0.60443477],
        [0.70523376, 0.47747142, 0.76880775, ..., 0.18718115,
         0.13212154, 0.59330825],
        [0.66877249, 0.43241425, 0.74952237, ..., 0.17527797,
         0.13212154, 0.59330825],
        ...,
        [0.66243138, 0.35588433, 0.70824082, ..., 0.31877043,
         0.14053173, 0.72702438],
        [0.67669886, 0.36940147, 0.7284298 , ..., 0.29483323,
         0.16521975, 0.7437339 ],
        [0.70047794, 0.39193006, 0.73762069, ..., 0.25101374,
         0.14867064, 0.71585829]],

       [[0.70523376, 0.47747142, 0.76880775, ..., 0.18718115,
         0.13212154, 0.59330825],
        [0.66877249, 0.43241425, 0.74952237, ..., 0.17527797,
         0.13212154, 0.59330825],
        [0.72267178, 0.48648284, 0.7862343 , ..., 0.16337475,
         0.13212154, 0.56266085],
        ...,
        [0.67669886, 0.36940147, 0.7284298 , ..., 0.29483323,
         0.16521975, 0.7437339 ],
        [0.7

In [189]:

from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dropout

model = Sequential()
model.add(LSTM(100,activation='relu',input_shape=(96, 7),return_sequences=True))
# model.add(LSTM(5,activation='relu',input_shape=(96, 7),return_sequences=True))

# model.add(LSTM(30,activation='relu',input_shape=(96, 7),return_sequences=True))
# model.add(Dropout(0.1))
# model.add(TimeDistributed(Dense(7)))

# model.add(LSTM(200, activation='relu', input_shape=(96, 7)))
# model.add(RepeatVector(96))
# model.add(LSTM(200, activation='relu', return_sequences=True))
# model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(7)))


model.compile(loss='mae', optimizer='adam')
hst = model.fit(train_X, train_y, epochs=15, batch_size=8, validation_data=(val_X, val_y), verbose=1, shuffle=False)

# #拟合神经网络
# history = model.fit(train_X, train_y, epochs=25, batch_size=16, validation_data=(val_X, val_y), verbose=0, shuffle=False)
# #画出学习过程
# p1 = pyplot.plot(history.history['loss'], color='blue', label='train')
# p2 = pyplot.plot(history.history['val_loss'], color='yellow',label='val')

# n_neuron = 50
# n_batch = 16
# n_epoch = 30
# repeats = 5
# inv_yhat_list = []
# inv_y_list = []
# # 
# model_list = fit_lstm(train_X,train_y,val_X,val_y, n_neuron, n_batch, n_epoch,repeats=repeats)
# for i in range(len(model_list)):
#     model = model_list[i]
#     inv_yhat = lstm_predict(model, data_prepare)[0]
#     inv_y = lstm_predict(model, data_prepare)[1]
#     inv_yhat_list.append(inv_yhat)
#     inv_y_list.append(inv_y)

Epoch 1/15
  42/1068 [>.............................] - ETA: 16s - loss: 0.1931

1068/1068 [==============================] - 23s 21ms/step - loss: 0.0406 - val_loss: 0.0782
Epoch 2/15
1068/1068 [==============================] - 26s 24ms/step - loss: 0.0121 - val_loss: 0.0425
Epoch 3/15
1068/1068 [==============================] - 23s 21ms/step - loss: 0.0077 - val_loss: 0.0193
Epoch 4/15
1068/1068 [==============================] - 23s 21ms/step - loss: 0.0058 - val_loss: 0.0146
Epoch 5/15
1068/1068 [==============================] - 24s 23ms/step - loss: 0.0048 - val_loss: 0.0124
Epoch 6/15
1068/1068 [==============================] - 24s 23ms/step - loss: 0.0039 - val_loss: 0.0118
Epoch 7/15
1068/1068 [==============================] - 19s 18ms/step - loss: 0.0031 - val_loss: 0.0110
Epoch 8/15
1068/1068 [==============================] - 15s 14ms/step - loss: 0.0031 - val_loss: 0.0111
Epoch 9/15
1068/1068 [==============================] - 16s 15ms/step - loss: 0.0029 - val_loss: 0.0102
Epoch 10/15
1068/1068 [==============================] - 17s 16ms/step - lo

In [190]:
v = model.predict(test_X)

93/93 [==============================] - 1s 10ms/step


In [200]:

pyplot.figure()

test_y=test_y.reshape(test_y.shape[0],96,7)

# pyplot.plot(test_y[3213][:,6])
# pyplot.plot(test_y[3214][:,6])

wind = 1714
iddx = 6

combined_line = list(    test_y[wind-96][:, iddx])  + list(test_y[wind][:, iddx])
pyplot.plot([None]*96+   list(v[wind][:,iddx]))        #2016   3214
pyplot.plot(combined_line)
pyplot.show()
